In [124]:
import pandas as pd
import numpy as np
import json
import re

#Plotting
import matplotlib.pyplot as plt
import FCPython 

#Statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression


import os
import glob

In [125]:
odds1718 = pd.read_csv('17_18_odds.csv', delimiter = ';')
odds1718.head(10)

,Spieltag,Datum,Heim,Gast,Ergebnis,Toto,1,0,2
0,34,5/12/18,TSG Hoffenheim,BVB 09 Dortmund,3:1,1,"2,34","4,2","2,94"
1,34,5/12/18,Hertha BSC Berlin,RB Leipzig,2:6,2,"4,5","4,1","1,84"
2,34,5/12/18,VfL Wolfsburg,1.FC Köln,4:1,1,"1,66","4,2",6
3,34,5/12/18,1.FSV Mainz 05,SV Werder Bremen,1:2,2,"2,66","3,8","2,74"
4,34,5/12/18,Hamburger SV,Mönchengladbach,2:1,1,"2,3",4,"3,1"
5,34,5/12/18,FC Schalke 04,Eintracht Frankfurt,1:0,1,"2,34","3,55","3,35"
6,34,5/12/18,SC Freiburg,FC Augsburg,2:0,1,"2,12","3,3","4,3"
7,34,5/12/18,Bayer Leverkusen,Hannover 96,3:2,1,"1,2","7,6",24
8,34,5/12/18,Bayern München,VfB Stuttgart,1:4,2,"1,14","8,6",40
9,33,5/5/18,1.FC Köln,Bayern München,1:3,2,"14,5","6,2","1,29"


In [126]:
teams = odds1718["Heim"].unique()
sorted(teams)

['1.FC Köln',
 '1.FSV Mainz 05',
 'BVB 09 Dortmund',
 'Bayer Leverkusen',
 'Bayern München',
 'Eintracht Frankfurt',
 'FC Augsburg',
 'FC Schalke 04',
 'Hamburger SV',
 'Hannover 96',
 'Hertha BSC Berlin',
 'Mönchengladbach',
 'RB Leipzig',
 'SC Freiburg',
 'SV Werder Bremen',
 'TSG Hoffenheim',
 'VfB Stuttgart',
 'VfL Wolfsburg']

In [127]:
#dictionary aus Score DataFrame

df_scores = pd.read_csv('df_scores.csv', index_col =0)

teams = df_scores.team_home.unique()
teams = sorted(teams)
teams_dict = {}

for index, element in enumerate(teams):
    teams_dict[element] = index +1
 
 
teams_dictinv = {v: k for k, v in teams_dict.items()}
    
teams_dict

{'Augsburg': 1,
 'Bayer Leverkusen': 2,
 'Bayern München': 3,
 'Borussia Dortmund': 4,
 "Borussia M'gladbach": 5,
 'Eintracht Frankfurt': 6,
 'Freiburg': 7,
 'Hamburger SV': 8,
 'Hannover 96': 9,
 'Hertha BSC': 10,
 'Hoffenheim': 11,
 'Köln': 12,
 'Mainz 05': 13,
 'RB Leipzig': 14,
 'Schalke 04': 15,
 'Stuttgart': 16,
 'Werder Bremen': 17,
 'Wolfsburg': 18}

### Erstelle auch teamsdict für obigen DataFrame:

In [128]:
teams

['Augsburg',
 'Bayer Leverkusen',
 'Bayern München',
 'Borussia Dortmund',
 "Borussia M'gladbach",
 'Eintracht Frankfurt',
 'Freiburg',
 'Hamburger SV',
 'Hannover 96',
 'Hertha BSC',
 'Hoffenheim',
 'Köln',
 'Mainz 05',
 'RB Leipzig',
 'Schalke 04',
 'Stuttgart',
 'Werder Bremen',
 'Wolfsburg']

In [129]:
odds_dict = {'Augsburg': 'FC Augsburg',
 'Bayer Leverkusen': 'Bayer Leverkusen',
 'Bayern München': 'Bayern München',
 'Borussia Dortmund': 'BVB 09 Dortmund',
 "Borussia M'gladbach": 'Mönchengladbach',
 'Eintracht Frankfurt': 'Eintracht Frankfurt',
 'Freiburg': 'SC Freiburg',
 'Hamburger SV': 'Hamburger SV',
 'Hannover 96': 'Hannover 96',
 'Hertha BSC': 'Hertha BSC Berlin',
 'Hoffenheim': 'TSG Hoffenheim',
 'Köln': '1.FC Köln',
 'Mainz 05': '1.FSV Mainz 05',
 'RB Leipzig': 'RB Leipzig',
 'Schalke 04': 'FC Schalke 04',
 'Stuttgart': 'VfB Stuttgart',
 'Werder Bremen': 'SV Werder Bremen',
 'Wolfsburg': 'VfL Wolfsburg'}


odds_dictinv = {v: k for k, v in odds_dict.items()}

In [130]:
df_odds = odds1718.replace(odds_dictinv)
df_odds = df_odds.replace(teams_dict)

In [131]:
df_scores = df_scores.replace(teams_dict)
df_scores

,team_home,team_away,ordresp
0,3,16,5.0
1,11,4,1.0
2,10,14,5.0
3,7,1,1.0
4,15,6,2.0
...,...,...,...
301,10,16,1.0
302,8,1,2.0
303,13,9,4.0
304,18,4,5.0


In [132]:
df_scores["odds_home"] = 0

In [133]:
df_odds

,Spieltag,Datum,Heim,Gast,Ergebnis,Toto,1,0,2
0,34,5/12/18,11,4,3:1,1,"2,34","4,2","2,94"
1,34,5/12/18,10,14,2:6,2,"4,5","4,1","1,84"
2,34,5/12/18,18,12,4:1,1,"1,66","4,2",6
3,34,5/12/18,13,17,1:2,2,"2,66","3,8","2,74"
4,34,5/12/18,8,5,2:1,1,"2,3",4,"3,1"
...,...,...,...,...,...,...,...,...,...
301,1,8/19/17,8,1,1:0,1,"2,3","3,4","3,6"
302,1,8/19/17,13,9,0:1,2,"1,92","3,7","4,6"
303,1,8/19/17,18,4,0:3,2,"4,1","3,95","1,95"
304,1,8/19/17,15,14,2:0,1,"2,94","3,5","2,64"


In [134]:
df_odds["1"]

0      2,34
1       4,5
2      1,66
3      2,66
4       2,3
       ... 
301     2,3
302    1,92
303     4,1
304    2,94
305    1,26
Name: 1, Length: 306, dtype: object

In [135]:
df_scores[df_scores["team_home"] == 11 & (df_scores["team_away"] == 12)]

,team_home,team_away,ordresp,odds_home
284,1,12,1.0,0


In [136]:
for index, match in df_scores.iterrows():
    home_odds = df_odds[(df_odds["Heim"] == match["team_home"]) & (df_odds["Gast"] == match["team_away"])]["1"]
    away_odds = df_odds[(df_odds["Heim"] == match["team_home"]) & (df_odds["Gast"] == match["team_away"])]["2"]
    draw_odds = df_odds[(df_odds["Heim"] == match["team_home"]) & (df_odds["Gast"] == match["team_away"])]["0"]
    
    
    df_scores.at[index, "odds_home"] = home_odds.item()
    df_scores.at[index, "odds_away"] = away_odds.item()
    df_scores.at[index, "odds_draw"] = draw_odds.item()
    
df_scores 

,team_home,team_away,ordresp,odds_home,odds_away,odds_draw
0,3,16,5.0,"1,14",40,"8,6"
1,11,4,1.0,"2,34","2,94","4,2"
2,10,14,5.0,"4,5","1,84","4,1"
3,7,1,1.0,"2,12","4,3","3,3"
4,15,6,2.0,"2,34","3,35","3,55"
...,...,...,...,...,...,...
301,10,16,1.0,"2,04","4,1","3,65"
302,8,1,2.0,"2,3","3,6","3,4"
303,13,9,4.0,"1,92","4,6","3,7"
304,18,4,5.0,"4,1","1,95","3,95"


In [137]:
df_scores["odds_home"] = pd.to_numeric(df_scores["odds_home"].str.replace(",","."))
df_scores["odds_away"] = pd.to_numeric(df_scores["odds_away"].str.replace(",","."))
df_scores["odds_draw"] = pd.to_numeric(df_scores["odds_draw"].str.replace(",","."))

In [138]:
df_scores

,team_home,team_away,ordresp,odds_home,odds_away,odds_draw
0,3,16,5.0,1.14,40.00,8.60
1,11,4,1.0,2.34,2.94,4.20
2,10,14,5.0,4.50,1.84,4.10
3,7,1,1.0,2.12,4.30,3.30
4,15,6,2.0,2.34,3.35,3.55
...,...,...,...,...,...,...
301,10,16,1.0,2.04,4.10,3.65
302,8,1,2.0,2.30,3.60,3.40
303,13,9,4.0,1.92,4.60,3.70
304,18,4,5.0,4.10,1.95,3.95


In [139]:
#df_scores[["odds_home", "odds_draw", "odds_away"]].to_csv("sorted_odds.csv", index=False)

## benötigen nur Quote für eingetretenes Ergebnis zunächst 

In [140]:
df_scores

,team_home,team_away,ordresp,odds_home,odds_away,odds_draw
0,3,16,5.0,1.14,40.00,8.60
1,11,4,1.0,2.34,2.94,4.20
2,10,14,5.0,4.50,1.84,4.10
3,7,1,1.0,2.12,4.30,3.30
4,15,6,2.0,2.34,3.35,3.55
...,...,...,...,...,...,...
301,10,16,1.0,2.04,4.10,3.65
302,8,1,2.0,2.30,3.60,3.40
303,13,9,4.0,1.92,4.60,3.70
304,18,4,5.0,4.10,1.95,3.95


In [141]:
for i, match in df_scores.iterrows():
    
    if (match["ordresp"] == 5) or (match["ordresp"]==4): 
        df_scores.at[i,"oddsresp"] = match["odds_away"]
    elif (match["ordresp"]==3):
        df_scores.at[i,"oddsresp"] = match["odds_draw"]
    else: 
        df_scores.at[i,"oddsresp"] = match["odds_home"]
        
        

In [142]:
df_scores

,team_home,team_away,ordresp,odds_home,odds_away,odds_draw,oddsresp
0,3,16,5.0,1.14,40.00,8.60,40.00
1,11,4,1.0,2.34,2.94,4.20,2.34
2,10,14,5.0,4.50,1.84,4.10,1.84
3,7,1,1.0,2.12,4.30,3.30,2.12
4,15,6,2.0,2.34,3.35,3.55,2.34
...,...,...,...,...,...,...,...
301,10,16,1.0,2.04,4.10,3.65,2.04
302,8,1,2.0,2.30,3.60,3.40,2.30
303,13,9,4.0,1.92,4.60,3.70,4.60
304,18,4,5.0,4.10,1.95,3.95,1.95


In [143]:
df_scores[["odds_home", "odds_draw", "odds_away", "oddsresp"]].to_csv("sorted_odds.csv", index=False)